In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Flatten, RepeatVector, TimeDistributed, InputLayer, Lambda
from tensorflow.keras import backend as K
from tensorflow.keras.initializers import RandomUniform, Initializer, Constant

In [2]:
class InitCentersRandom(Initializer):

    def __init__(self, X):
        self.X = X

    def __call__(self, shape, dtype=None):
        assert shape[1] == self.X.shape[1]
        idx = np.random.randint(self.X.shape[0], size=shape[0])
        return self.X[idx, :]

class RBFLayer(keras.layers.Layer):

    def __init__(self, output_dim, initializer=None, betas=1.0, **kwargs):
        self.output_dim = output_dim
        self.init_betas = betas
        if not initializer:
            self.initializer = RandomUniform(0.0, 1.0)
        else:
            self.initializer = initializer
        super(RBFLayer, self).__init__(**kwargs)

    def build(self, input_shape):

        self.centers = self.add_weight(name='centers',
                                       shape=(self.output_dim, input_shape[1]),
                                       initializer=self.initializer,
                                       trainable=True)
        self.betas = self.add_weight(name='betas',
                                     shape=(self.output_dim,),
                                     initializer=Constant(
                                         value=self.init_betas),
                                     # initializer='ones',
                                     trainable=True)

        super(RBFLayer, self).build(input_shape)

    def call(self, x):

        C = K.expand_dims(self.centers)
        H = K.transpose(C-K.transpose(x))
        return K.exp(-self.betas * K.sum(H**2, axis=1))

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

    def get_config(self):
        # have to define get_config to be able to use model_from_json
        config = {
            'output_dim': self.output_dim
        }
        base_config = super(RBFLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [3]:
data = 'collateral/train_mod.csv'
kval = 4
itertot = 40
itergd = 300
    
dataread = np.genfromtxt(data, delimiter=',')[1:,1:]
alldata = []
for i in range(len(dataread)):
    if np.isnan(dataread[i,-2]):
        continue
    alldata.append(dataread[i])


alldata = np.asarray(alldata)

#dividing data
trainparam = alldata[:600,1:]
trainlabel = alldata[:600,0]
testparam = alldata[600:,1:]
testlabel = alldata[600:,0]

std = np.zeros((len(trainparam[0]))).astype('float32')
rata = np.zeros((len(trainparam[0]))).astype('float32')
trainparamnorm = np.zeros(np.shape(trainparam))
testparamnorm = np.zeros(np.shape(testparam))
for i in range(len(trainparam[0])):
    std[i] = np.std(trainparam[:,i])
    rata[i] = np.mean(trainparam[:,i])
    trainparamnorm[:,i] = (trainparam[:,i] - rata[i]) / std[i]
    testparamnorm[:,i] = (testparam[:,i] - rata[i]) / std[i]

In [4]:
def calczscore(x):    
    output = x
    
    if not x.shape[0] is None:
        o = []
        for i in range(x.shape[0]):
            o.append([])
            for n in range(x[i].shape[0]):
                o[i].append((x[i][n] - rata[n]) / std[n])
        output = tf.reshape(o, shape=x.shape)
    return output

def generatemodel(numparam):
    model = Sequential()
    model.add(Flatten(input_shape=((numparam, 1))))
    model.add(RBFLayer(kval,
                      initializer=InitCentersRandom(calczscore(trainparam).numpy()),
                      betas=2.0))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [5]:
model = generatemodel(kval)

model.fit(trainparamnorm, trainlabel, batch_size=20, epochs=itergd, verbose=1, shuffle=True)

Epoch 1/300
30/30 [==============================] - 0s 4ms/step - loss: 0.6999 - accuracy: 0.4267
Epoch 2/300
30/30 [==============================] - 0s 5ms/step - loss: 0.6951 - accuracy: 0.5167
Epoch 3/300
30/30 [==============================] - 0s 4ms/step - loss: 0.6909 - accuracy: 0.5950
Epoch 4/300
30/30 [==============================] - 0s 4ms/step - loss: 0.6869 - accuracy: 0.6033
Epoch 5/300
30/30 [==============================] - 0s 4ms/step - loss: 0.6831 - accuracy: 0.6067
Epoch 6/300
30/30 [==============================] - 0s 5ms/step - loss: 0.6793 - accuracy: 0.6117
Epoch 7/300
30/30 [==============================] - 0s 4ms/step - loss: 0.6756 - accuracy: 0.6117
Epoch 8/300
30/30 [==============================] - 0s 4ms/step - loss: 0.6720 - accuracy: 0.6183
Epoch 9/300
30/30 [==============================] - 0s 5ms/step - loss: 0.6684 - accuracy: 0.6217
Epoch 10/300
30/30 [==============================] - 0s 4ms/step - loss: 0.6651 - accuracy: 0.6250
Epoch 11/

30/30 [==============================] - 0s 4ms/step - loss: 0.4875 - accuracy: 0.7800
Epoch 84/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4866 - accuracy: 0.7750
Epoch 85/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4857 - accuracy: 0.7750
Epoch 86/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4849 - accuracy: 0.7800
Epoch 87/300
30/30 [==============================] - 0s 3ms/step - loss: 0.4840 - accuracy: 0.7767
Epoch 88/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4837 - accuracy: 0.7667
Epoch 89/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4825 - accuracy: 0.7750
Epoch 90/300
30/30 [==============================] - 0s 3ms/step - loss: 0.4817 - accuracy: 0.7683
Epoch 91/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4809 - accuracy: 0.7733
Epoch 92/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4801 - accuracy: 0.7750
Epoch 93/300


30/30 [==============================] - 0s 4ms/step - loss: 0.4533 - accuracy: 0.7817
Epoch 165/300
30/30 [==============================] - 0s 5ms/step - loss: 0.4540 - accuracy: 0.7817
Epoch 166/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4531 - accuracy: 0.7850
Epoch 167/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4533 - accuracy: 0.7850
Epoch 168/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4530 - accuracy: 0.7883
Epoch 169/300
30/30 [==============================] - 0s 3ms/step - loss: 0.4530 - accuracy: 0.7833
Epoch 170/300
30/30 [==============================] - 0s 5ms/step - loss: 0.4530 - accuracy: 0.7817
Epoch 171/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4527 - accuracy: 0.7767
Epoch 172/300
30/30 [==============================] - 0s 3ms/step - loss: 0.4525 - accuracy: 0.7817
Epoch 173/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4523 - accuracy: 0.7817
Epoc

30/30 [==============================] - 0s 4ms/step - loss: 0.4467 - accuracy: 0.7933
Epoch 246/300
30/30 [==============================] - 0s 5ms/step - loss: 0.4467 - accuracy: 0.7917
Epoch 247/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4468 - accuracy: 0.7900
Epoch 248/300
30/30 [==============================] - 0s 5ms/step - loss: 0.4469 - accuracy: 0.7900
Epoch 249/300
30/30 [==============================] - 0s 5ms/step - loss: 0.4474 - accuracy: 0.7950
Epoch 250/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4465 - accuracy: 0.7900
Epoch 251/300
30/30 [==============================] - 0s 5ms/step - loss: 0.4466 - accuracy: 0.7917
Epoch 252/300
30/30 [==============================] - 0s 5ms/step - loss: 0.4466 - accuracy: 0.7917
Epoch 253/300
30/30 [==============================] - 0s 4ms/step - loss: 0.4464 - accuracy: 0.7867
Epoch 254/300
30/30 [==============================] - 0s 5ms/step - loss: 0.4466 - accuracy: 0.7833
Epoc

In [6]:
lifeprob = model.predict(testparamnorm)
print(lifeprob)
#determine biner accuracy
binpred = np.zeros((len(lifeprob)))
for i in range(len(lifeprob)):
    if lifeprob[i] > 0.5:
        binpred[i] = 1.
score = 0
for i in range(len(testlabel)):
    if binpred[i] == testlabel[i]:
        score += 1
accbin = float(score) / float(len(testlabel))
#determine brier score
brierscore = 0
for i in range(len(testlabel)):
    brierscore += (testlabel[i] - lifeprob[i])**2.0
brierscore = brierscore / float(len(testlabel))
for i in range(len(testlabel)):
    print(lifeprob[i], testlabel[i])
print(accbin, brierscore[0])

[[0.14226177]
 [0.7942637 ]
 [0.4192235 ]
 [0.11769417]
 [0.28212443]
 [0.09843999]
 [0.98414254]
 [0.0889515 ]
 [0.16207123]
 [0.96188027]
 [0.19241598]
 [0.92546797]
 [0.4775582 ]
 [0.10335788]
 [0.13517427]
 [0.08925435]
 [0.62421924]
 [0.67630213]
 [0.17626053]
 [0.5144363 ]
 [0.7797104 ]
 [0.5087481 ]
 [0.98127127]
 [0.50575006]
 [0.13213292]
 [0.13194397]
 [0.5046309 ]
 [0.25778627]
 [0.3369611 ]
 [0.36234227]
 [0.29578927]
 [0.13134241]
 [0.11886641]
 [0.9273324 ]
 [0.47646543]
 [0.11068642]
 [0.4884846 ]
 [0.15037882]
 [0.9178339 ]
 [0.6046053 ]
 [0.349041  ]
 [0.12269634]
 [0.10688809]
 [0.38991976]
 [0.5047877 ]
 [0.11886641]
 [0.9892609 ]
 [0.12642816]
 [0.0796923 ]
 [0.14549336]
 [0.5211115 ]
 [0.10825711]
 [0.49718428]
 [0.16568214]
 [0.08871838]
 [0.23689023]
 [0.91242814]
 [0.12115538]
 [0.40112662]
 [0.49055365]
 [0.3229537 ]
 [0.4177095 ]
 [0.8111741 ]
 [0.51113635]
 [0.41770867]
 [0.14230046]
 [0.17574504]
 [0.97766066]
 [0.1539559 ]
 [0.09456548]
 [0.16141   ]
 [0.29